In [ ]:
import os
from glob import glob

import numpy as np

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers, models

import keras
from keras.models import Model
from keras.layers import Add, Activation, Concatenate, Conv2D, Dropout
from keras.layers import Flatten, Input, GlobalAveragePooling2D, MaxPooling2D
import keras.backend as K
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
device_name = tf.test.gpu_device_name()

if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')

print('Found GPU at: {}'.format(device_name))

In [ ]:
IMAGE_SIZE = [256, 256]

In [ ]:
train_path = 'data/grayscale/train'
test_path = 'data/grayscale/test'

In [ ]:
def count(directory, counter=0):
    """returns number of files in dir and subdirs"""
    for pack in os.walk(directory):
        for f in pack[2]: counter += 1
    return directory + " : " + str(counter) + " files"

In [ ]:
print('total images for training :', count(train_path))
print('total images for validation :', count(test_path))

In [ ]:
folders = glob('data/grayscale/train/*')
print(len(folders))

In [ ]:
BATCH_SIZE = 32

train_generator = ImageDataGenerator(preprocessing_function = preprocess_input, validation_split = 0.1)

test_generator = ImageDataGenerator(preprocessing_function = preprocess_input)

In [ ]:
class_subset = sorted(os.listdir('data/grayscale/train'))

print(class_subset)

traingen = train_generator.flow_from_directory(train_path,
                                               target_size=(256, 256),
                                               class_mode='categorical',
                                               classes=class_subset,
                                               subset='training',
                                               batch_size=BATCH_SIZE,
                                               shuffle=True,
                                               seed=5)

validgen = train_generator.flow_from_directory(train_path,
                                               target_size=(256, 256),
                                               class_mode='categorical',
                                               classes=class_subset,
                                               subset='validation',
                                               batch_size=BATCH_SIZE,
                                               shuffle=True,
                                               seed=4)

testgen = test_generator.flow_from_directory(test_path,
                                             target_size=(256, 256),
                                             class_mode=None,
                                             classes=class_subset,
                                             batch_size=1,
                                             shuffle=False,
                                             seed=4)

In [ ]:
def SqueezeNet_11(input_shape, nb_classes, dropout_rate=None, compression=1.0):
    """
    Creating a SqueezeNet of version 1.1

    Arguments:
        input_shape  : shape of the input images e.g. (224,224,3)
        nb_classes   : number of classes
        dropout_rate : defines the dropout rate that is accomplished after last fire module (default: None)
        compression  : reduce the number of feature-maps

    Returns:
        Model        : Keras model instance
    """

    input_img = Input(shape=input_shape)

    x = Conv2D(int(64*compression), (3,3), activation='relu', strides=(2,2), padding='same', name='conv1')(input_img)

    x = MaxPooling2D(pool_size=(3,3), strides=(2,2), name='maxpool1')(x)

    x = create_fire_module(x, int(16*compression), name='fire2')
    x = create_fire_module(x, int(16*compression), name='fire3')

    x = MaxPooling2D(pool_size=(3,3), strides=(2,2), name='maxpool3')(x)

    x = create_fire_module(x, int(32*compression), name='fire4')
    x = create_fire_module(x, int(32*compression), name='fire5')

    x = MaxPooling2D(pool_size=(3,3), strides=(2,2), name='maxpool5')(x)

    x = create_fire_module(x, int(48*compression), name='fire6')
    x = create_fire_module(x, int(48*compression), name='fire7')
    x = create_fire_module(x, int(64*compression), name='fire8')
    x = create_fire_module(x, int(64*compression), name='fire9')

    if dropout_rate:
        x = Dropout(dropout_rate)(x)

    # Creating last conv10
    x = output(x, nb_classes)

    return Model(inputs=input_img, outputs=x)

In [ ]:
def output(x, nb_classes):
    x = Conv2D(nb_classes, (1,1), strides=(1,1), padding='valid', name='conv10')(x)
    x = GlobalAveragePooling2D(name='avgpool10')(x)
    x = Activation("softmax", name='softmax')(x)
    return x

In [ ]:
def create_fire_module(x, nb_squeeze_filter, name, use_bypass=False):
    """
    Creates a fire module

    Arguments:
        x                 : input
        nb_squeeze_filter : number of filters of squeeze. The filtersize of expand is 4 times of squeeze
        use_bypass        : if True then a bypass will be added
        name              : name of module e.g. fire123

    Returns:
        x                 : returns a fire module
    """

    nb_expand_filter = 4 * nb_squeeze_filter
    squeeze    = Conv2D(nb_squeeze_filter,(1,1), activation='relu', padding='same', name='%s_squeeze'%name)(x)
    expand_1x1 = Conv2D(nb_expand_filter, (1,1), activation='relu', padding='same', name='%s_expand_1x1'%name)(squeeze)
    expand_3x3 = Conv2D(nb_expand_filter, (3,3), activation='relu', padding='same', name='%s_expand_3x3'%name)(squeeze)

    axis = get_axis()
    x_ret = Concatenate(axis=axis, name='%s_concatenate'%name)([expand_1x1, expand_3x3])

    if use_bypass:
        x_ret = Add(name='%s_concatenate_bypass'%name)([x_ret, x])

    return x_ret

In [ ]:
def get_axis():
    axis = -1 if K.image_data_format() == 'channels_last' else 1
    return axis

In [ ]:
SqueezeNet = SqueezeNet_11(input_shape=IMAGE_SIZE + [3], nb_classes=10)

In [ ]:
SqueezeNet.summary()